In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import time
import os

if not os.path.exists("htmls"):
    os.makedirs("htmls")

def get_proxy():
    proxy_list = [{'https': '144.76.68.98:3128'}, {'https': '138.68.73.59:32574'},
                  {'https': '144.76.68.98:3128'}, {'https': '144.76.189.74:8080'},
                  {'https': '138.201.159.242:3128'}, {'https': '91.194.121.98:57205'},
                  {'https': '5.128.199.11:35478'}, {'https': '195.9.61.22:33094'},
                  {'http': '142.93.128.104:8080'}, {'https': '78.10.44.10:8080'},
                  {'https': '195.189.89.122:31562'}, {'https': '212.87.248.11:40762'},
                  {'https': '88.199.82.72:35765'}, {'https': '194.181.3.53:35333'},
                  {'https': '185.56.210.102:30280'}, {'https': '188.114.66.110:43773'},
                  {'https': '85.232.248.9:8080'}, {'https': '192.166.221.51:48789'},
                  {'https': '109.206.223.186:59874'}
                  ]
    for proxy in proxy_list:
        print(proxy)
        yield proxy


def ok_html(html):
    if html is not None:
        try:
            soup = BeautifulSoup(html.text, 'html.parser')
            elem = soup.find(class_='title-info-metadata-item').text
        except:
            elem = None
    else:
        elem = None
    return elem is not None


def find_items(substrings, items):

    for item in items:
        flag = False
        for substr in substrings:
            if item == substr:
                flag = True
        if flag == False:
            return False
    return True

def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

def combine_price(substrings):
    res = ''
    for substr in substrings:
        if is_number(substr):
            res += substr
    return res


pd.set_option('display.max_columns', 300)
columns = ['size', 'class', 'address', 'price', 'descr', 'metros', 'url']


proxies = {'http': '84.52.74.194:8080'}
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' +
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' +
                         'Chrome/42.0.2311.90 Safari/537.36'}

parsed_urls = []
try:
    df = pd.DataFrame.from_csv('result.csv')
    for i in df['url']:
        parsed_urls.append(str(i))
except:
    df = pd.DataFrame(columns=columns)

print(parsed_urls)

with open('failed.txt', 'a+') as failedfile:
    with open('avito_offices.txt', 'r') as file:
        prefix = file.readline()[:-1]

        for i in range(3000):
            suffix = file.readline()[:-1]

            url = prefix + suffix
            print(url)

            if url in parsed_urls:
                print('parsed before')
            else:
                counter = 1

                proxies = get_proxy()
                try:
                    html = requests.get(url)
                except:
                    html = None
                while (not ok_html(html)) & (counter < 10):
                    counter += 1    
                    try:
                        html = requests.get(url, proxies=next(proxies))
                    except:
                        html = None
                
                if counter < 10:

                    soup = BeautifulSoup(html.text, 'html.parser')

                    file_html = open('htmls/' + suffix + '.html', 'w+', encoding='utf-8')
                    file_html.write(html.text)
                    file_html.close()

                    data = [None] * len(columns)
                    try:
                        try:
                            elem = soup.find(class_='page_title-count').text
                        except:
                            elem = soup.find(class_='title-info-metadata-item').text
                            page_id = elem.split()[1][:-1]

                            # альтернативный способ найти площадь
                            # elem = soup.find(class_='title-info-title-text').text
                            # size = float(elem.split()[-2])
                            # data[0] = size

                            elem = soup.find(class_='item-params').text.split()
                            size = float(elem[1])
                            data[0] = size
                            if len(elem) > 3:
                               class_ = elem[5]
                               data[1] = class_

                            address = soup.find(class_='item-map-address').text
                            data[2] = re.sub('\n', '', address)

                            items = ('месяц', '₽', 'м²')

                            elem = soup.find(class_='price-value-string js-price-value-string').text.split()
                            if find_items(elem, items):
                                data[3] = combine_price(elem)
                            else:
                                elems = soup.find_all(class_='price-value-prices-list-item')
                                for elem in elems:
                                    elem = elem.text.split()
                                    if find_items(elem, items):
                                        data[3] = combine_price(elem)
                            try:
                                description = soup.find(class_='item-description-text').text
                            except:
                                description = soup.find(class_='item-description-html').text
                            data[4] = re.sub('\n', '', description)

                            elems = soup.find_all(class_='item-map-metro')
                            str_ = ""
                            for elem in elems:
                                str_ += re.sub('\n', '', elem.text) + '.'
                            data[5] = str_

                            data[6] = url

                            ser = pd.Series(name=page_id, data=data, index=columns)
                            df = df.append(ser)

                            parsed_urls.append(url)
                            df.to_csv('result.csv')
                    except:
                        failedfile.write(url + '\n')
                        print('error   ' + url)

                    time.sleep(1)
                else:
                    failedfile.write(url + '\n')
                    print('error   ' + url)
                    time.sleep(1)

df.to_csv('result.csv')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:78: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


['https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/sdam_ofisnoe_pomeschenie_102.20_m_1562570154', 'https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/sdacha_ofisnogo_pomescheniya_73m_1411320446', 'https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_19_m_1162111084', 'https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_40_m_615188087', 'https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_165_m_1212858338', 'https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/gotovyy_ofis_110_kv_m_1606021203', 'https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofis_u_metro_mebel_bez_komissii_705787321', 'https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_105_m_987864891', 'https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofis_s_yur.adresom_sao_20_m_941909351', 'https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofis_v_bts_rechnoy_vokzal_104.9_m_1128968420', 'https://www.avito

https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/prestizhnyy_kovorking_s_yuridicheskim_adresom_721025372
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/kovorking_v_bts_sadovaya_plaza_725844932
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofis_pod_klyuch_pryamaya_arenda_19_m_581391519
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/arenda_rabochih_mest_po_programme_businessworld_728624604
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofis_v_arendu_ot_6_chelovek_v_tsentre_moskvy_873013344
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/rabochee_mesto_po_karte_businessworld_gold_727042926
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_v_biznes-tsentre_voentorg_727047442
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofis_na_1_rabochee_mesto_s_mebelyu_906858374
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofis_s_mebelyu_otkrytaya_planirovka_ot_6_chelovek_604162640
https://www.avito.ru/mosk

{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
{'https': '144.76.68.98:3128'}
{'https': '144.76.189.74:8080'}
{'https': '138.201.159.242:3128'}
{'https': '91.194.121.98:57205'}
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/sdam_ofisnoe_pomeschenie_53.3_m_1593988727
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/87_m_ofis_na_dmitrovskom_sh._ot_sobstvennika_1095882575
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/torgovo-ofisnoe_pomeschenie_ot_60-300_m_1465944151
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofis_ot_sobstvennika_35.1_m_1016080190
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_15_m_1265081989
{'https': '144.76.68.98:3128'}
{'https': '138.68.7

{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_14_m_302317345
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_124.1_m_1134079890
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_172_m_1590454191
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/sdam_ofisnoe_pomeschenie_28.7_m_1495535310
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/sdam_ofisnoe_pomeschenie_57.4_m_1379549625
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofis_klassa_a_558.10_m_1064363797
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:

...

{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
{'https': '144.76.68.98:3128'}
{'https': '144.76.189.74:8080'}
{'https': '138.201.159.242:3128'}
{'https': '91.194.121.98:57205'}
{'https': '5.128.199.11:35478'}
{'https': '195.9.61.22:33094'}
{'http': '142.93.128.104:8080'}
error   https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/arenda_pomescheniya_1231095505
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_38.75_m_1661392099
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
{'https': '144.76.68.98:3128'}
{'https': '144.76.189.74:8080'}
{'https': '138.201.159.242:3128'}
{'https': '91.194.121.98:57205'}
{'https': '5.128.199.11:35478'}
{'https': '195.9.61.22:33094'}
{'http': '142.93.128.104:8080'}
error   https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_38.75_m_1661392099
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnyy_osobnyak_klassa_v_1046_m_1674691377
{'https': '144.76.68.98:3128'}


{'https': '195.9.61.22:33094'}
{'http': '142.93.128.104:8080'}
error   https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/gotovyy_ofis_132_kv_m_945643998
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_233.2_m_1636694847
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_332_m_1718755765
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
{'https': '144.76.68.98:3128'}
{'https': '144.76.189.74:8080'}
{'https': '138.201.159.242:3128'}
{'https': '91.194.121.98:57205'}
{'https': '5.128.199.11:35478'}
{'https': '195.9.61.22:33094'}
{'http': '142.93.128.104:8080'}
error   https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_332_m_1718755765
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_50.4_m_1256619442
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_20.3_m_1100132847
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
{'https': '14

https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_1742_m_bts_klass_v_yuao_782474316
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
{'https': '144.76.68.98:3128'}
{'https': '144.76.189.74:8080'}
{'https': '138.201.159.242:3128'}
{'https': '91.194.121.98:57205'}
{'https': '5.128.199.11:35478'}
{'https': '195.9.61.22:33094'}
{'http': '142.93.128.104:8080'}
error   https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_1742_m_bts_klass_v_yuao_782474316
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_87_m_908000138
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
{'https': '144.76.68.98:3128'}
{'https': '144.76.189.74:8080'}
{'https': '138.201.159.242:3128'}
{'https': '91.194.121.98:57205'}
{'https': '5.128.199.11:35478'}
{'https': '195.9.61.22:33094'}
{'http': '142.93.128.104:8080'}
error   https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_87_m_908000138
https:

{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
{'https': '144.76.68.98:3128'}
{'https': '144.76.189.74:8080'}
{'https': '138.201.159.242:3128'}
{'https': '91.194.121.98:57205'}
{'https': '5.128.199.11:35478'}
{'https': '195.9.61.22:33094'}
{'http': '142.93.128.104:8080'}
error   https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_468_m_1666060537
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_18.5_m_1713271058
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
{'https': '144.76.68.98:3128'}
{'https': '144.76.189.74:8080'}
{'https': '138.201.159.242:3128'}
{'https': '91.194.121.98:57205'}
{'https': '5.128.199.11:35478'}
{'https': '195.9.61.22:33094'}
{'http': '142.93.128.104:8080'}
error   https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_18.5_m_1713271058
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_65_m_i_drugie_pl_1008559680
{'https': '144.76.68.98

{'https': '195.9.61.22:33094'}
{'http': '142.93.128.104:8080'}
error   https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_120_m_1544823411
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/ofisnoe_pomeschenie_272.5_m_625407053
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/sdam_ofisnoe_pomeschenie_102.1_m_1636957818
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
{'https': '144.76.68.98:3128'}
{'https': '144.76.189.74:8080'}
{'https': '138.201.159.242:3128'}
{'https': '91.194.121.98:57205'}
{'https': '5.128.199.11:35478'}
{'https': '195.9.61.22:33094'}
{'http': '142.93.128.104:8080'}
error   https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/
https://www.avito.ru/moskva/kommercheskaya_nedvizhimost/
{'https': '144.76.68.98:3128'}
{'https': '138.68.73.59:32574'}
{'https': '144.76.68.98:3128'}
{'https': '144.76.189.74:8080'}
{'https': '138.201.159.242:3128'}
{'https': '91.1

In [25]:
with open('final_result.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv)